# Holoviews

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from skimage import io

from pathlib import Path
import holoviews as hv

from nima import nima

cm = plt.cm.inferno_r
hv.notebook_extension()

# tdata = Path("../../tests/data/")
tdata = Path("../../tests/data/")
fp = tdata / "1b_c16_15.tif"

In [ ]:
channels = ["G", "R", "C"]
dim, n_ch, times = nima.read_tiff(fp, channels)

dimm = nima.d_median(dim)
f = nima.d_show(dimm, cmap=cm)

In [ ]:
%%opts Image [aspect=512/512] 

%%opts Image.Cyan style(cmap=plt.cm.Blues)
%%opts Image.Green style(cmap=plt.cm.Greens)
%%opts Image.Red style(cmap=plt.cm.Reds)

chans = hv.Image(dim['C'][0], group='cyan') \
    + hv.Image(dim['G'][0], group='green') \
    + hv.Image(dim['R'][0], group='red')

chans

In [ ]:
c = [(i, hv.Image(im)) for i, im in enumerate(dim["C"])]
c = hv.HoloMap(c, kdims=["Frame"])
g = [(i, hv.Image(im)) for i, im in enumerate(dim["G"])]
g = hv.HoloMap(g, kdims=["Frame"])
r = [(i, hv.Image(im)) for i, im in enumerate(dim["R"])]
r = hv.HoloMap(r, kdims=["Frame"])

In [ ]:
%%output holomap='auto'
%%opts Image style(cmap='viridis')
(c + g).select(Frame={0,5,6,7,10,30}).cols(2)

In [ ]:
c[::20].overlay("Frame")

In [ ]:
wl = hv.Dimension("excitation wavelength", unit="nm")
c = c.add_dimension(wl, 1, 458)
g = g.add_dimension(wl, 1, 488)
r = r.add_dimension(wl, 1, 561)

channels = c.clone()
channels.update(g)
channels.update(r)

In [ ]:
%%opts Image style(cmap='viridis')
%%output size=300
channels[::5].grid(['Frame', 'excitation wavelength'])

In [ ]:
t = [(i, hv.Image(im)) for i, im in enumerate(dim["C"])]

In [ ]:
hv.HoloMap([(i, hv.Image(im)) for i, im in enumerate(dim["C"])], kdims=["frame"])

In [ ]:
hv.NdLayout(
    {
        k: hv.HoloMap(
            [(i, hv.Image(im)) for i, im in enumerate(dim[k])], kdims=["frame"]
        )
        for k in dim
    },
    kdims=["channels"],
)[::4]

In [ ]:
%%opts Image (cmap='viridis') 
%%opts Image.A [aspect=2]
im = hv.Image(dim["G"][1], bounds=(0, 0, 512, 512))
im2 = hv.Image(dim['C'][1], bounds=(0, 0, 512, 512))
im3 = hv.Image(dimm['C'][1], bounds=(0, 0, 512, 512))
((im * hv.HLine(y=350)) + im.sample(y=350) + (im2 * hv.HLine(y=150)) + im2.sample(y=150) * im3.sample(y=150)).cols(3)